In [76]:
import torch as t
import math
import numpy as np
import matplotlib.pyplot as plt
import sys

# define problem:
def f(x):
    return t.tensor([[x[0]**2 + (x[1]-3)**2]])

def df(x):
    return t.tensor([[2*x[0],
                    2*(x[1]-3)]])

def g(x):
    return t.tensor([[x[1]**2-2*x[0],
                    (x[1]-1)**2+5*x[0]-15]])

def dg(x):
    return t.tensor([[-2,2*x[1]],
                    [5,2*(x[1]-1)]])

def solve_activeset(c,b,a,W):
    M_L = t.cat((W, a), 0)
    M_R = t.transpose(t.cat((a, t.zeros(a.size(dim=0),a.size(dim=0))), 1),0,1)
    M = t.cat((M_L, M_R), 1)
    U = t.transpose(t.cat((c, b), 1),0,1)
    sol = t.linalg.solve(M, U)

    S = sol[[0,1]]
    MU = sol[2:]
    return [S, MU]

def QP_solver(df0,g0,dg0):
    t.set_printoptions(precision=10)
    keepCal = True
    active = [] # [] for empty; [0] for g1; [1] for g2; [0,1] for g1 & g2
    i=0

    while keepCal:
        i = i+1

        mu0 = t.zeros(g0.size())
        g=g0[:,active]
        dg=dg0[active,:]

        [s, mu] = solve_activeset(-df0,-g,dg,H)

        n_digits = 12
        mu = (mu * 10**n_digits).round() / (10**n_digits)

        mu0[:,active]=t.transpose(mu,0,1)

        g_check = t.matmul(dg0,s)+t.transpose(g0,0,1)

        n_digits = 12
        g_check = (g_check * 10**n_digits).round() / (10**n_digits)

        mu_check = False

        if t.numel(mu)==0:
            mu_check = True
        elif t.min(mu)>0:
            mu_check = True
        else:
            remove=t.argmin(mu0).tolist()
            active.remove(remove)

        if t.max(g_check) <= 0:
            if mu_check:
                keepCal = False
        else:
            add=t.argmax(g_check).tolist()
            active.append(add)
        active = sorted(list(dict.fromkeys(active)))
        if i  > 50:
            sys.exit("QP solver too many iterations")
    return [s, mu0]

def line_search(F, dF, G, dG, x, s, mu_k, w_k):
    z = 0.1 # scale factor on current gradient: [0.01, 0.3]
    b = 0.8 # scale factor on backtracking: [0.1, 0.8]
    a = 1. # maximum step length

    D = t.squeeze(t.transpose(s,0,1)) # direction of search

    # merit function:
    w_k_1 = t.maximum(abs(mu_k), 0.5*(w_k+abs(mu_k)))

    count = 0
    while count<1000:

        phi_a = f(x+a*D) + t.matmul(w_k_1,t.transpose(t.maximum(t.zeros(g(x).size()),g(x+a*D)),0,1))

        phi0 = F + t.matmul(w_k_1,t.transpose(t.maximum(t.zeros(g(x).size()),G),0,1))

        dphi0 = t.matmul(dF,D) + t.matmul(w_k_1,t.transpose(((G>0)*1.)*t.matmul(dG,D),0,1))

        psi_a = phi0 + z*a*dphi0

        if phi_a<psi_a:
            break
        else:
            a = a*b
            count = count + 1

    return [a, w_k]

# Initialization:
x0 = t.tensor([1.,1.])
x_k = x0

# Initialization of the Hessian matrix
H = t.eye(t.numel(x_k))

# Initialization of the Lagrange multipliers
LM_k = t.zeros(g(x_k).size())   # Start with zero Lagrange multiplier estimates
# Initialization of the weights in merit function
w = t.zeros(g(x_k).size())   # Start with zero weights

# Set the termination criterion
opt_eps = 1e-3
G_norm = t.norm(df(x_k)+t.matmul(LM_k,dg(x_k)))

while G_norm>opt_eps:
    # solve QP
    [s_k, LM_k_1] = QP_solver(df(x_k),g(x_k),dg(x_k)) # return search direction (k) and new Lagrangian Multiplier(k+1)

    # LineSearch
    [a, w] = line_search(f(x_k), df(x_k), g(x_k), dg(x_k), x_k, s_k, LM_k, w)

    # Update the current solution using the step
    dx = t.transpose(a*s_k,0,1)  # Step for x

    x_k_1 = t.squeeze(x_k + dx)   # Update x using the step


    # Update Hessian using BFGS:

    # Compute y_k:
    y_k =df(x_k_1) + t.matmul(LM_k_1,dg(x_k_1)) - df(x_k) -  t.matmul(LM_k_1,dg(x_k))

    # Compute theta:
    Left = t.matmul(dx,t.transpose(y_k,0,1))
    Right = t.matmul(t.matmul(dx,H),t.transpose(dx,0,1))

    if Left >= 0.2*Right:
        theta = 1
    else:
        theta = 0.8*Right/(Right-Left)

    # Compute  dg_k:
    dg_k = theta*y_k + (1-theta)*t.matmul(dx,H)

    # Compute new Hessian:
    H = H + t.transpose(dg_k,0,1)*dg_k/t.sum(dg_k*dx) - t.transpose(t.matmul(dx,H),0,1)*t.matmul(dx,H)/Right

    # Update termination criterion:
    G_norm = t.norm(df(x_k_1)+t.matmul(LM_k_1,dg(x_k_1)))

    # Update x and lagrangian multipliers:
    x_k =x_k_1

    LM_k = LM_k_1
print(x_k)

tensor(4.4721360207)
Start solve QP:
1
M, tensor([[1., 0.],
        [0., 1.]])
U, tensor([[-2.],
        [ 4.]])
sol: tensor([[-2.],
        [ 4.]])
mu: tensor([], size=(0, 1))
dg0 and g0: tensor([[-2.,  2.],
        [ 5.,  0.]]) tensor([[ -1., -10.]])
g_check: tensor([[ 11.],
        [-20.]])
active: [0]
2
M, tensor([[ 1.,  0., -2.],
        [ 0.,  1.,  2.],
        [-2.,  2.,  0.]])
U, tensor([[-2.],
        [ 4.],
        [ 1.]])
sol: tensor([[0.7500000000],
        [1.2500000000],
        [1.3750000000]])
mu: tensor([[1.3750000000]])
dg0 and g0: tensor([[-2.,  2.],
        [ 5.,  0.]]) tensor([[ -1., -10.]])
g_check: tensor([[ 0.0000000000],
        [-6.2500000000]])
active: [0]
Start Line Search:
1 tensor([[1., 0.],
        [0., 1.]])
tensor([[0.9985482693, 0.6008710265],
        [0.6008710265, 4.3894777298]])
tensor([1.7500000000, 2.2500000000])
tensor(4.7471208572)
Start solve QP:
1
M, tensor([[0.9985482693, 0.6008710265],
        [0.6008710265, 4.3894777298]])
U, tensor([[-3.50

In [74]:
import torch as t
a = t.tensor([ 0.4,  1.5,  1.4, -0.7])
print(a.round())

tensor([ 0.,  2.,  1., -1.])
